In [1]:
import os

In [2]:
pwd


'c:\\Users\\Robin Aluma\\Desktop\\Car_Price_Pred\\Resources'

In [3]:
os.chdir('../')

In [4]:
pwd


'c:\\Users\\Robin Aluma\\Desktop\\Car_Price_Pred'

In [5]:

from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np
import sys
import pandas as pd

In [6]:

from src.Car_Price_Pred.utils.common import load_models,load_parameters,read_yaml,create_directories
from dataclasses import dataclass
from pathlib import Path
from src.Car_Price_Pred.constants import *
from sklearn.model_selection import train_test_split
from Exceptions import CustomException

In [7]:
@dataclass
class ModelEvaluationConfig():
    root_url: Path
    evaluation_data: Path
    best_model_path: Path
    decision_tree: str
    lasso: str
    linear_regression: str
    neighbors: str
    svm: str
    random_forest: str
    

In [8]:
class ModelEvaluationConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH):
        self.config = read_yaml(config_file_path)
        
        create_directories([self.config.Artifacts_root])
        
    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_url])
        model_evaluation_config = ModelEvaluationConfig(
            root_url = config.root_url,
            evaluation_data = config.evaluation_data,
            best_model_path = config.best_model_path,
            decision_tree = config.decision_tree,
            lasso = config.lasso,
            linear_regression = config.linear_regression,
            neighbors = config.neighbors,
            svm = config.svm,
            random_forest = config.random_forest
        )
        
        return model_evaluation_config
    

In [9]:
class ModelEvaluation():
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config
        
    def get_xy_testdata(self):
        test_data = pd.read_csv(self.config.evaluation_data)
        
        test_data.fillna(test_data.mean(),inplace=True)
        
        self.x_test = test_data.iloc[:,:-1]
        self.y_test = test_data.iloc[:,-1]
        
        models = [
            load_models(self.config.decision_tree),
            load_models(self.config.lasso),
            load_models(self.config.linear_regression),
            load_models(self.config.neighbors),
            load_models(self.config.svm),
            load_models(self.config.random_forest)]
        
        for model in models:
            self.y_predict = model.predict(self.x_test)
            
    def calculate_metrics(self):
        y_true = self.y_test
        y_pred = self.y_predict
        
        r_score = r2_score(y_true,y_pred)
        mae = mean_absolute_error(y_true,y_pred)
        mse = mean_squared_error(y_true,y_pred)
        rmse = np.sqrt(mse)
        
        
        print(f"r2_score:====={r_score}=====")
        print(f"mean_absolute_error:====={mae}====")
        print(f"mean_squared_error:====={mse}====")
        print(f"root_mean_squared_error:====={rmse}=====")
        
        

In [10]:
class ModelEvaluationPipeline:
    def __init__(self):
        pass
    def main(self):
        try:
            
            config = ModelEvaluationConfigurationManager()
            get_model_evaluation_config = config.get_model_evaluation_config()
            model_evaluation = ModelEvaluation(config = get_model_evaluation_config)
            model_evaluation.get_xy_testdata()
            model_evaluation.calculate_metrics()
        except Exception as e:
            
            raise CustomException(e,sys)
        

In [11]:
if __name__ == '__main__':
    try:
        obj = ModelEvaluationPipeline()
        obj.main()
    except Exception as e:
        raise CustomException(e,sys)

[ 2025-01-22 10:50:42,565 : common : INFO : Creates directory for path;artifacts in file paths :['artifacts'] ]
[ 2025-01-22 10:50:42,567 : common : INFO : Creates directory for path;artifacts/model_evaluation in file paths :['artifacts/model_evaluation'] ]
r2_score:=====0.9076820780777544=====
mean_absolute_error:=====6.307357249457032====
mean_squared_error:=====120.15825005990713====
root_mean_squared_error:=====10.961671864269023=====


c:\Users\Robin Aluma\Desktop\Car_Price_Pred\Car_price\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\Robin Aluma\Desktop\Car_Price_Pred\Car_price\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
c:\Users\Robin Aluma\Desktop\Car_Price_Pred\Car_price\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
c:\Users\Robin Aluma\Desktop\Car_Price_Pred\Car_price\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
c:\Users\Robin Aluma\Desktop\Car_Price_Pred\Car_price\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SVR was fitted 